In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

C:\Users\haho6\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from keras.preprocessing.image import load_img, img_to_array
images = []
for i in range(470):
    image = load_img('./DATA/'+str(i)+'.jpeg',target_size=(64,64))
    image = img_to_array(image)
    image.shape
    image = image / 256
    images.append(image-0.5)
    
    
images = np.array(images)

Using TensorFlow backend.


In [3]:

def sample_z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [4]:
X = tf.placeholder(tf.float32, shape=[None, 64 , 64 , 3])

In [5]:
W1 = tf.Variable(tf.random_normal([3,3,3,32],stddev=0.02))
W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.02))
W3 = tf.Variable(tf.random_normal([3,3,64,256],stddev=0.02))
theta_D = [W1,W2,W3]

def discriminaster(inputs):
    
    H1 = tf.nn.conv2d(inputs,W1, strides=[1,2,2,1], padding="SAME")
    H1 = tf.layers.batch_normalization(H1)
    H1 = tf.nn.relu(H1)


    H2 = tf.nn.conv2d(H1,W2, strides=[1,2,2,1], padding="SAME")
    H2 = tf.nn.relu(H2)


    H3 = tf.nn.conv2d(H2,W3, strides=[1,2,2,1], padding="SAME")
    H3 = tf.nn.relu(H3)

    out = tf.reshape(H3,[-1,16384])
    out = tf.layers.dense(out,1)
    return out

In [6]:
print(discriminaster(X))
print()
print(theta_D)

Tensor("dense/BiasAdd:0", shape=(?, 1), dtype=float32)

[<tf.Variable 'Variable:0' shape=(3, 3, 3, 32) dtype=float32_ref>, <tf.Variable 'Variable_1:0' shape=(3, 3, 32, 64) dtype=float32_ref>, <tf.Variable 'Variable_2:0' shape=(3, 3, 64, 256) dtype=float32_ref>]


In [7]:
Z = tf.placeholder(tf.float32, shape=[None, 50])

In [8]:
G_W1 = tf.Variable(tf.random_normal([50, 4*4*1024]))
G_W2 = tf.Variable(tf.random_normal([8, 8, 512, 1024]))
G_W3 = tf.Variable(tf.random_normal([16, 16, 256, 512]))
G_W4 = tf.Variable(tf.random_normal([32, 32, 128, 256]))
G_W5 = tf.Variable(tf.random_normal([64, 64, 3, 128]))

theta_G = [G_W1, G_W2, G_W3, G_W4]
def generate(Z):
    
    H0 = tf.nn.relu(tf.matmul(Z,G_W1))
    H0 = tf.reshape(H0,[-1,4,4,1024])
    #
    H1 = tf.nn.conv2d_transpose(H0,strides=[1,2,2,1],filter=G_W2
                                ,output_shape=(5,8,8,512), padding="SAME")
    H1 = tf.nn.relu(H1)
    #
    H2 = tf.nn.conv2d_transpose(H1,strides=[1,2,2,1],filter=G_W3
                                ,output_shape=(5,16,16,256), padding="SAME")
    H2 = tf.nn.relu(H2)
    #
    H3 = tf.nn.conv2d_transpose(H2,strides=[1,2,2,1],filter=G_W4
                                ,output_shape=(5,32,32,128), padding="SAME")
    H3 = tf.nn.tanh(H3)
    #
    H4 = tf.nn.conv2d_transpose(H3,strides=[1,2,2,1],filter=G_W5
                                ,output_shape=(5,64,64,3), padding="SAME")
    #
    
    return H4

In [104]:
generate(Z)

<tf.Tensor 'conv2d_transpose_57:0' shape=(5, 64, 64, 3) dtype=float32>

In [9]:
G_sample = generate(Z)
D_real = discriminaster(X)
D_fake = discriminaster(G_sample)

In [10]:
D_loss = tf.reduce_mean(D_real) - tf.reduce_mean(D_fake)
G_loss = -tf.reduce_mean(D_fake)

D_solver = (tf.train.RMSPropOptimizer(learning_rate=0.01)
            .minimize(-D_loss, var_list=theta_D))
G_solver = (tf.train.RMSPropOptimizer(learning_rate=0.01)
            .minimize(G_loss, var_list=theta_G))

In [11]:
clip_D = [p.assign(tf.clip_by_value(p, -0.01, 0.01)) for p in theta_D]

In [12]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [141]:
sess.run(generate(Z),feed_dict={Z:sample_z(10,50)})

array([[[[7.72788544e+08, 4.23117760e+08, 2.31649328e+08],
         [0.00000000e+00, 0.00000000e+00, 7.14531840e+08],
         [0.00000000e+00, 0.00000000e+00, 3.05075424e+08],
         ...,
         [0.00000000e+00, 3.46659552e+08, 0.00000000e+00],
         [4.34801536e+08, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

        [[3.56726304e+08, 0.00000000e+00, 3.31549632e+08],
         [0.00000000e+00, 1.28744144e+08, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 4.58930400e+08],
         ...,
         [0.00000000e+00, 6.29045280e+07, 5.43911744e+08],
         [0.00000000e+00, 9.42445696e+08, 0.00000000e+00],
         [3.93260704e+08, 3.78018816e+08, 1.04743912e+08]],

        [[3.69661160e+07, 0.00000000e+00, 4.03493248e+08],
         [0.00000000e+00, 0.00000000e+00, 2.16036720e+08],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         ...,
         [0.00000000e+00, 3.44933376e+08, 0.00000000e+00],
         [

In [33]:
for i in range(2000):
    noise = sample_z(5, 50)
    _, D_loss_curr, _ = sess.run(
            [D_solver, D_loss, clip_D],
            feed_dict={X: images[55:60], Z: noise}
        )
    _, G_loss_curr = sess.run(
        [G_solver, G_loss],
        feed_dict={Z: noise}
    )
    print(str(i)+"-> G : "+str(G_loss_curr)+"\tD :"+str(D_loss_curr))

0-> G : 13916.011	D :1613.1208
1-> G : 13995.974	D :1628.4758
2-> G : 13752.62	D :1631.0802
3-> G : 14022.356	D :1658.6174
4-> G : 13680.502	D :1648.6934
5-> G : 13564.797	D :1650.1812
6-> G : 13562.38	D :1655.0331
7-> G : 13200.255	D :1637.0607
8-> G : 13415.43	D :1652.7413
9-> G : 13244.65	D :1644.6704
10-> G : 13059.214	D :1635.7771
11-> G : 13036.256	D :1636.5844
12-> G : 13239.591	D :1646.8376
13-> G : 13297.178	D :1651.5181
14-> G : 13226.591	D :1648.4387
15-> G : 13114.373	D :1643.4564
16-> G : 13229.896	D :1652.5934
17-> G : 13158.8125	D :1645.4885
18-> G : 12868.056	D :1633.0197
19-> G : 12977.926	D :1636.2881
20-> G : 13266.19	D :1654.5776
21-> G : 13016.359	D :1633.6742
22-> G : 13132.997	D :1639.4716
23-> G : 13133.13	D :1639.7949
24-> G : 12910.278	D :1628.6603
25-> G : 13143.172	D :1642.267
26-> G : 13243.662	D :1649.4839
27-> G : 13250.761	D :1648.4692
28-> G : 12927.924	D :1631.0059
29-> G : 12937.26	D :1629.9269
30-> G : 12822.973	D :1627.8994
31-> G : 13009.364	D :163

254-> G : 14194.5	D :1668.5725
255-> G : 13802.997	D :1653.0762
256-> G : 13785.016	D :1662.3115
257-> G : 13973.357	D :1675.4193
258-> G : 13711.919	D :1667.7583
259-> G : 13953.25	D :1664.8971
260-> G : 13888.367	D :1671.4788
261-> G : 13665.479	D :1660.2947
262-> G : 13741.347	D :1673.6395
263-> G : 13470.034	D :1663.9641
264-> G : 12959.513	D :1636.1976
265-> G : 12860.534	D :1628.2474
266-> G : 12990.8955	D :1637.2211
267-> G : 12929.141	D :1632.2413
268-> G : 13004.106	D :1640.0071
269-> G : 12994.625	D :1639.5116
270-> G : 13091.338	D :1642.6797
271-> G : 12916.713	D :1633.6049
272-> G : 12979.495	D :1636.0281
273-> G : 12700.071	D :1621.3888
274-> G : 13004.012	D :1635.56
275-> G : 13027.894	D :1638.2784
276-> G : 12910.79	D :1630.3009


KeyboardInterrupt: 

In [34]:
print(G_loss_curr)
print(D_loss_curr)

12910.79
1625.1039


In [35]:
samples = sess.run(G_sample, feed_dict={Z:
                                        sample_z(5, 50)})
for i in range(5):
    windowUp(samples[i])

In [16]:
def windowUp(arr):
    import cv2
    frame = cv2.cvtColor(np.array(arr), cv2.COLOR_BGR2RGB)
    cv2.imshow("test", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
